# LangChain Chat with Your Data

<!--img src="https://github.com/ksm26/LangChain-Chat-with-Your-Data/raw/main/images/L1-Document_loading.png" width=500-->

## Retrieval augmented generation
 
검색 증강 생성(retrieval augmented generation, RAG)에서 LLM은 실행의 일부로 외부 데이터 세트에서 컨텍스트 문서를 검색합니다.

이는 특정 문서(예: PDF, 비디오 세트 등)에 대해 질문하려는 경우에 유용합니다.

![overview.jpeg](overview.jpeg)

## Document Loading

오디오 및 비디오를 포함한 다양한 데이터 소스를 처리하기 위해 LangChain에서 제공하는 80개 이상의 고유한 로더(loader)에 액세스합니다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbgnoaH%2FbtsITVVTJFb%2FIk46LYmguStdSwiIDDJXa1%2Fimg.png" width="650">

<!-- ![Document Loaders](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbgnoaH%2FbtsITVVTJFb%2FIk46LYmguStdSwiIDDJXa1%2Fimg.png) -->
* 로더(loader)는 데이터 액세스 및 변환의 세부 사항을 처리합니다.
  - Accessing
    - Web Sites
    - Data Bases
    - YouTube
    - arXiv
    - ...
  - Data Types
    - PDF
    - HTML
    - JSON
    - Word, PowerPoint,
    - ...
   
      
* Returns a list of "Document' objects:
```
[
Document(page_content="MachineLearning-Lecture01 \nInstructor (Andrew Ng): Okay. Good morning, Welcome to CS229 ...', metadata={'source': 'docs/cs229_lectures/MacchineLearning-Lecture01.pdf', 'page':0})
...
Document(page_content="[End of Audio] \nDuration: 69 minutes', metadata={'source': 'docs/cs229_lectures/MacchineLearning-Lecture01.pdf', 'page':21})
]
```

In [94]:
import os
import openai
import sys
#sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

## PDFs

Andrew Ng의 유명한 CS229 코스에서 PDF [transcript](https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture01.pdf)을 로드해 봅시다!
이러한 문서는 자동 전사(transcription)의 결과이므로 단어와 문장이 예기치 않게 분리되는 경우가 있습니다.

In [97]:
#! pip install pypdf 
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf")
pages = loader.load()

Each page is a `Document`.

A `Document` contains text (`page_content`) and `metadata`.

In [99]:
len(pages)

22

In [101]:
page = pages[0]

In [103]:
print(page.page_content[0:500])

MachineLearning-Lecture01  
Instructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine 
learning class. So what I wanna do today is just spend a little time going over the logistics 
of the class, and then we'll start to talk a bit about machine learning.  
By way of introduction, my name's Andrew Ng and I'll be instructor for this class. And so 
I personally work in machine learning, and I've worked on it for about 15 years now, and 
I actually think that machine learning is the 


In [107]:
pages[5].metadata

{'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 5}

## YouTube

In [36]:
# Old Obsolete
# from langchain.document_loaders.generic import GenericLoader
# from langchain.document_loaders.parsers import OpenAIWhisperParser
# from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
# # ! pip install yt_dlp
# # ! pip install pydub
# url="https://www.youtube.com/watch?v=jGwO_UgTS7I"
# save_dir="docs/youtube/"
# loader = GenericLoader(
#     YoutubeAudioLoader([url],save_dir),
#     OpenAIWhisperParser()
# )
# docs = loader.load()

In [110]:
#!pip install pytube
from pytube import extract
from langchain_community.document_loaders import YoutubeLoader

video_url = 'https://youtu.be/jGwO_UgTS7I?si=ukkUH2Yk-ut9BVGB'
video_id = extract.video_id(video_url)
loader = YoutubeLoader(video_id)
docs = loader.load()

In [112]:
len(docs)

1

In [40]:
docs[0].page_content[0:500]

"Welcome to CS229 Machine Learning. Uh, some of you know that this class has been taught at Stanford for a long time. And this is often the course that, um, I most look forward to teaching each year because this is where we've helped I think, several generations of Stanford students become experts in machine learning, go on to build many of their products and services and startups that I'm sure many of you are pre- or all of you are using, uh, uh, today. Um, so what I want to do today was spend s"

In [114]:
video_url = 'https://youtu.be/CQ0tUe9tu5g?si=pD6Lnf5l5oDyXvMe'
video_id = extract.video_id(video_url)
loader = YoutubeLoader(video_id,language='ko')
docs = loader.load()
docs[0].page_content[0:500]

'머신 러닝이란 뭘까요 그이 비디오에서는 그게 뭔지 정의를 한번 배워보고 언제 적용하고 싶은지 알게 될 겁니다 함께 살펴보도록 하겠습니다 자 여기아서 세미엘이라는 그 학자가 제시한 머신러닝의 정의가 쓰여 있는데 다음과 같습니다 그는 어떻게 정의했냐면요 머신 러닝은 이와 정의했습니다 feeld of the that gives computers that ability explicitely 즉 명시적으로 믹스 플레이 세팅과 명시적으로 직접 프로그래밍하지 않고도 컴퓨터한테 학습할 수 있는 능력을 제공하는 연구 분야다라고 이렇게 정의했는데요 사실이 세뮤엘이라는이 분의 명성은 한 1950년대에 지금 여기 보여지고 있는요 그림과 같은 체커 게임 프로그램을 만드셨습니다 근데이 프로그램이 놀라운 점은요 자신이 체커를 잘 두는 선수가 아니었는데도 불구하고 그가 한 일은 컴퓨터가 스스로 수만개의 게임을 플레이 하도록 이렇게 프로그래밍함으로써이 프로그램 그 게임 프로그램을 만들었다고 보시면 됩니다 결국에 '

## URLs

In [85]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Artificial_intelligence")

In [51]:
docs = loader.load()

In [116]:
len(docs)

1

In [53]:
print(docs[0].page_content[:500])





Artificial intelligence - Wikipedia


























Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload file



















Search











Search






















Appearance
















Donate

Create account

Log in








Personal tools





Donate Create account Log in





		Pages


## Notion

[이 사이트](https://yolospace.notion.site/Blendle-s-Employee-Handbook-e31bff7da17346ee99f531087d8b133f)와 같은 Notion site의 예로서 [여기](https://python.langchain.com/docs/integrations/document_loaders/notion/#1-create-a-notion-table-database)의 단계를 따라 :

* 페이지를 자신의 Notion 공간에 복제하고 `Markdown/CSV`로 내보냅니다.
* 압축을 풀고 Notion 페이지에 대한 마크다운 파일이 포함된 폴더로 저장합니다.

<img src="image.png" width="500">
<!-- ![image.png](./image.png) -->

In [122]:
from langchain_community.document_loaders import NotionDirectoryLoader
loader = NotionDirectoryLoader("docs/Notion_DB")

In [124]:
docs = loader.load()

In [126]:
len(docs)

48

In [63]:
print(docs[0].page_content[0:200])

# Blendle's Employee Handbook

This is a living document with everything we've learned working with people while running a startup. And, of course, we continue to learn. Therefore it's a document that


In [65]:
docs[0].metadata

{'source': "docs\\Notion_DB\\Blendle's Employee Handbook 1f049c3723744841b572dfd7781e71c4.md"}